In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
pwd

'/content'

In [ ]:
root_path = 'gdrive/My Drive/capstone_project/input/recognizing-faces-in-the-wild'

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir(root_path))

# Any results you write to the current directory are saved as output.

['test', 'train', '.ipynb_checkpoints', 'kernel_submission.csv', 'Data Cleaning and Preprocessing.ipynb', '.DS_Store', 'sample_submission.csv', 'combinations.txt', 'Untitled.ipynb', 'open-kimono-kinship-notebook-0-792lb.ipynb', 'train_relationships.csv', 'test.zip', 'train.zip', 'df_final.csv', 'kinship-detection-with-vgg16.ipynb']


Necassary Imports

In [ ]:
!pip install git+https://github.com/rcmalli/keras-vggface.git

  Cloning https://github.com/rcmalli/keras-vggface.git to /tmp/pip-req-build-7oljf2nb
  Running command git clone -q https://github.com/rcmalli/keras-vggface.git /tmp/pip-req-build-7oljf2nb
  Stored in directory: /tmp/pip-ephem-wheel-cache-jody4tgl/wheels/36/07/46/06c25ce8e9cd396dabe151ea1d8a2bc28dafcb11321c1f3a6d
Successfully built keras-vggface


In [ ]:
import h5py
from collections import defaultdict
from glob import glob
from random import choice, sample

import cv2
import numpy as np
import pandas as pd
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.layers import Input, Dense, Flatten, GlobalMaxPool2D, GlobalAvgPool2D, Concatenate, Multiply, Dropout, Subtract, Add, Conv2D
from keras.models import Model
from keras.optimizers import Adam
from keras_vggface.utils import preprocess_input
from keras_vggface.vggface import VGGFace

Using TensorFlow backend.


In [ ]:
train_file_path = root_path + "/train_relationships.csv"
train_folders_path = root_path + "/train/"
val_famillies = "F09"
train_folders_path

'gdrive/My Drive/capstone_project/input/recognizing-faces-in-the-wild/train/'

In [ ]:
train_file_path

'gdrive/My Drive/capstone_project/input/recognizing-faces-in-the-wild/train_relationships.csv'

In [ ]:
train_folders_path

'gdrive/My Drive/capstone_project/input/recognizing-faces-in-the-wild/train/'

In [ ]:
all_images = glob(root_path + "/train/" + "*/*/*.jpg")
len(all_images)

12429

Validation set consists only family 9

In [ ]:
train_images = [x for x in all_images if val_famillies not in x]
val_images = [x for x in all_images if val_famillies in x]

In [ ]:
train_person_to_images_map = defaultdict(list)

In [ ]:
ppl = [x.split("/")[-3] + "/" + x.split("/")[-2] for x in all_images]

In [ ]:
for x in train_images:
    train_person_to_images_map[x.split("/")[-3] + "/" + x.split("/")[-2]].append(x)

val_person_to_images_map = defaultdict(list)

for x in val_images:
    val_person_to_images_map[x.split("/")[-3] + "/" + x.split("/")[-2]].append(x)

relationships = pd.read_csv(train_file_path)
relationships = list(zip(relationships.p1.values, relationships.p2.values))
relationships = [x for x in relationships if x[0] in ppl and x[1] in ppl]

train = [x for x in relationships if val_famillies not in x[0]]
val = [x for x in relationships if val_famillies in x[0]]

In [ ]:
def read_img(path):
    img = cv2.imread(path)
    img = np.array(img).astype(np.float)
    return preprocess_input(img, version=2)


def gen(list_tuples, person_to_images_map, batch_size=16):
    ppl = list(person_to_images_map.keys())
    while True:
        batch_tuples = sample(list_tuples, batch_size // 2)
        labels = [1] * len(batch_tuples)
        while len(batch_tuples) < batch_size:
            p1 = choice(ppl)
            p2 = choice(ppl)

            if p1 != p2 and (p1, p2) not in list_tuples and (p2, p1) not in list_tuples:
                batch_tuples.append((p1, p2))
                labels.append(0)

        for x in batch_tuples:
            if not len(person_to_images_map[x[0]]):
                print(x[0])

        X1 = [choice(person_to_images_map[x[0]]) for x in batch_tuples]
        X1 = np.array([read_img(x) for x in X1])

        X2 = [choice(person_to_images_map[x[1]]) for x in batch_tuples]
        X2 = np.array([read_img(x) for x in X2])

        yield [X1, X2], labels


def baseline_model():
    input_1 = Input(shape=(224, 224, 3))
    input_2 = Input(shape=(224, 224, 3))

    base_model = VGGFace(model='resnet50', include_top=False)

    for layer in base_model.layers[:-3]:
        layer.trainable = True

    x1 = base_model(input_1)
    x2 = base_model(input_2)


    merged_add = Add()([x1, x2])
    merged_sub = Subtract()([x1,x2])
    
    merged_add = Conv2D(100 , [1,1] )(merged_add)
    merged_sub = Conv2D(100 , [1,1] )(merged_sub)
    
    merged = Concatenate(axis=-1)([merged_add, merged_sub])
    
    merged = Flatten()(merged)

    merged = Dense(100, activation="relu")(merged)
    merged = Dropout(0.3)(merged)
    merged = Dense(25, activation="relu")(merged)
    merged = Dropout(0.3)(merged)
    out = Dense(1, activation="sigmoid")(merged)

    model = Model([input_1, input_2], out)

    model.compile(loss="binary_crossentropy", metrics=['acc'], optimizer=Adam(0.00001))

    model.summary()

    return model


file_path = "vgg_face.h5"

checkpoint = ModelCheckpoint(file_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

reduce_on_plateau = ReduceLROnPlateau(monitor="val_acc", mode="max", factor=0.1, patience=20, verbose=1)

callbacks_list = [checkpoint, reduce_on_plateau]

model = baseline_model()

W0627 10:01:58.822360 139859425564544 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0627 10:01:58.859759 139859425564544 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0627 10:01:58.867912 139859425564544 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0627 10:01:58.900207 139859425564544 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0627 10:01:58.902811 1398594255

94699520/94694792 [==============================] - 6s 0us/step


W0627 10:02:22.428784 139859425564544 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W0627 10:02:22.491325 139859425564544 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0627 10:02:22.501400 139859425564544 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
vggface_resnet50 (Model)        multiple             23561152    input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
add_17 (Add)                    (None, 1, 1, 2048)   0           vggface_resnet50[1][0]           
          

In [ ]:
model.fit_generator(gen(train, train_person_to_images_map, batch_size=16), use_multiprocessing=True,
                    validation_data=gen(val, val_person_to_images_map, batch_size=16), epochs=10, verbose=1,
                    workers = 4, callbacks=callbacks_list, steps_per_epoch=200, validation_steps=100)

Epoch 1/10


/usr/local/lib/python3.6/dist-packages/keras/engine/training_generator.py:47: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the`keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'


200/200 [==============================] - 591s 3s/step - loss: 1.2386 - acc: 0.5375 - val_loss: 0.7275 - val_acc: 0.5962

Epoch 00001: val_acc improved from -inf to 0.59625, saving model to vgg_face.h5
Epoch 2/10
200/200 [==============================] - 275s 1s/step - loss: 0.7998 - acc: 0.5759 - val_loss: 0.6804 - val_acc: 0.6138

Epoch 00002: val_acc improved from 0.59625 to 0.61375, saving model to vgg_face.h5
Epoch 3/10
200/200 [==============================] - 181s 905ms/step - loss: 0.7131 - acc: 0.5850 - val_loss: 0.6547 - val_acc: 0.6244

Epoch 00003: val_acc improved from 0.61375 to 0.62438, saving model to vgg_face.h5
Epoch 4/10
200/200 [==============================] - 137s 686ms/step - loss: 0.7023 - acc: 0.6028 - val_loss: 0.6414 - val_acc: 0.6425

Epoch 00004: val_acc improved from 0.62438 to 0.64250, saving model to vgg_face.h5
Epoch 5/10
200/200 [==============================] - 112s 559ms/step - loss: 0.6677 - acc: 0.6131 - val_loss: 0.6265 - val_acc: 0.6388

Epo

In [ ]:
test_path = root_path + "/test/"


def chunker(seq, size=32):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))


from tqdm import tqdm

submission = pd.read_csv(root_path + '/sample_submission.csv')

predictions = []

for batch in tqdm(chunker(submission.img_pair.values)):
    X1 = [x.split("-")[0] for x in batch]
    X1 = np.array([read_img(test_path + x) for x in X1])

    X2 = [x.split("-")[1] for x in batch]
    X2 = np.array([read_img(test_path + x) for x in X2])

    pred = model.predict([X1, X2]).ravel().tolist()
    predictions += pred

submission['is_related'] = predictions

submission.to_csv("vgg_face.csv", index=False)

64it [31:47, 23.07s/it]